In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
CRYPTO_TARGET='NONE' 
SS_VER='SS_VER_1_1'

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../hardware/victims/firmware/hawk
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

In [ ]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/hawk/project-{}.hex".format(PLATFORM))

In [ ]:
from random import randint
from tqdm import tnrange

def get_traces(N):
    traces = []
    x = []
    
    for i in tnrange(N, desc='Capturing traces'):
        text = bytearray([0]*16)
        text[0] = randint(0,255)

        scope.arm()
    
        target.simpleserial_write('p', text)
    
        ret = scope.capture()
        if ret:
            print("Target timed out!")
    
        response = target.simpleserial_read('r', 16)
        x.append(response[0])
    
        traces.append(scope.get_last_trace())
    
        scope.arm()
    
    return (traces, x)
    

In [ ]:
ktp = cw.ktp.Basic()
key, text = ktp.next()
target.set_key(key)
traces, x = get_traces(10000)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
for t in traces[:10]:
    plt.plot(t[:2000], linewidth=0.5)
plt.show()

In [ ]:
import numpy as np

x = np.array(x)
traces = np.array(traces)
np.save('x.npy', x)
np.save('traces.npy', traces)

In [ ]:
scope.dis()
target.dis()